# LangChain Integration Tutorial 🦜🔗

**Let's get started on using Arize with LangChain!** ✨

LangChain is a cutting-edge framework that facilitates the development of powerful applications driven by large language models. It follows two key principles: agenticity and data awareness. LangChain provides a wide range of modules that enable language models to connect with diverse data sources and interact seamlessly with their surroundings.

Use Arize and LangChain together to effectively monitor the performance of your LLM agents, identify areas that require improvement, and make prompt engineering decisions about your LLM applications. With Arize and LangChain together, data scientists and machine learning engineers can ensure that their LLM applications are running at peak efficiency, enabling them to deliver improved results and drive greater value for their organizations.

## Running This Notebook
1. Step through each section below, pressing play on the code blocks to run the cells.
2. Log in your browser to Arize App
3. Copy and paste your Arize Space and API key
4. For your own LLM Applications, define the arize_callback within your LLM definitions.

# Step 0. Install Dependencies, Import Libraries, Use GPU 📚

Import LangChain, Arize, and Arize CallBack Handler for integration between two tools. 

⚠️ Use a GPU to save time generating embeddings. Click on 'Runtime', select 'Change Runtime Type' and
select 'GPU'. 

In [ ]:
!pip3 install -q langchain 
!pip3 install -q arize 
!pip3 install -q 'arize[AutoEmbeddings]'
!pip3 install -q openai

In [ ]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.arize_callback import ArizeCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.llms import OpenAI

# Step 1. Import and Setup Arize Client

The first step is to setup our Arize client. After that we will log the data from LangChain driven application into Arize.

Retrieve your Arize `API_KEY` and `SPACE_KEY` from your Space Settings page, and paste them in the set-up section below. 

We will also set up some metadata and the `ArizeCallBackHandler` to use while logging.




<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [ ]:
SPACE_KEY = "YOUR_SPACE_KEY"
API_KEY = "YOUR_API_KEY"

if SPACE_KEY == "YOUR_SPACE_KEY" or API_KEY == "YOUR_API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")

# Define callback handler for Arize
arize_callback = ArizeCallbackHandler(
    model_id="llm-langchain-demo",
    model_version="1.0",
    SPACE_KEY=SPACE_KEY,
    API_KEY=API_KEY,
)

In [ ]:
#Put your OPENAI API Key here!
%env OPENAI_API_KEY="YOUR OPEN API KEY"   

## Step 2: Define LLM with ArizeCallBack Handler
Use the callback handler we defined above within the LLM with LangChain.

In [ ]:
manager = CallbackManager([StdOutCallbackHandler(), arize_callback])
llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)

## Step 3: Test LLM Responses and Logging into Arize
Use some simple prompts to test if the LLM works properly and each prompt-response pair is logged into Arize with embeddings. 

In [ ]:
llm_result = llm.generate(
    [
        "Tell me an interesting fact about pandas.",
        "Explain the concept of overfitting in 2 sentences.",
    ]
)

## Step 4: Test LLM Chain and Agents with Arize Integration

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True, callback_manager=manager
)

test_prompts = [
    {
        "input": "documentary about pandas who are about be extinct because of global warming"
    },
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]
overall_chain.apply(test_prompts)

### Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
- [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/)
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
- [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/)
- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
